In [1]:
## You need to install NI-DAQmx runtime first (x64)
## Then, use pip install nidaqmx to install the library## 

In [1]:
from nidaqmx.system import System
sys = System.local()
print([d.name for d in sys.devices])      # Result should be something like ['Dev1']
print("AI channels:", [c.name for c in sys.devices[0].ai_physical_chans])

['Dev1']
AI channels: ['Dev1/ai0', 'Dev1/ai1', 'Dev1/ai2', 'Dev1/ai3', 'Dev1/ai4', 'Dev1/ai5', 'Dev1/ai6', 'Dev1/ai7']


In [ ]:
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, AcquisitionType

DEV_CH   = "Dev1/ai0"                  # Channel Name
TERMINAL = TerminalConfiguration.DIFF  # Differential Reading (AI0 + and -)
FS       = 1000                        # Sample Rate [S/s]
N_PER    = 1000                        # Counts
CAL      = 100.0                       # e.g.: 1 V = 100 N

with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan(
        DEV_CH,
        min_val=-10.0, max_val=10.0,
        terminal_config=TERMINAL
    )
    task.timing.cfg_samp_clk_timing(FS, sample_mode=AcquisitionType.CONTINUOUS)

    print("Read Data (Ctrl+C to shut down)")
    while True:
        vals = task.read(number_of_samples_per_channel=N_PER)
        mean_v = sum(vals)/len(vals)
        forceN = mean_v * CAL
        print(f"{mean_v:.5f} V  →  {forceN:.3f} N")


Read Data (Ctrl+C to shut down)
0.06428 V  →  6.428 N
0.06433 V  →  6.433 N
0.06428 V  →  6.428 N
0.06413 V  →  6.413 N
0.06426 V  →  6.426 N
0.06427 V  →  6.427 N
0.06426 V  →  6.426 N
1.37928 V  →  137.928 N
2.39001 V  →  239.001 N
2.82854 V  →  282.854 N
1.75005 V  →  175.005 N
0.06411 V  →  6.411 N
0.06409 V  →  6.409 N
0.06394 V  →  6.394 N
0.06397 V  →  6.397 N
0.06392 V  →  6.392 N
0.06383 V  →  6.383 N
0.06419 V  →  6.419 N
0.06386 V  →  6.386 N
0.06387 V  →  6.387 N
0.06384 V  →  6.384 N
0.06378 V  →  6.378 N
0.06392 V  →  6.392 N
0.06384 V  →  6.384 N
0.06392 V  →  6.392 N
0.06394 V  →  6.394 N


KeyboardInterrupt: 

In [14]:
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, AcquisitionType

DEV_CH   = "Dev1/ai0"                  # Channel Name
TERMINAL = TerminalConfiguration.DIFF  # Differential Reading (AI0 + and -)
FS       = 1000                        # Sample Rate [S/s]
N_PER    = 1000                        # Counts
CAL      = 20.306                       # e.g.: 1 V = 100 N
REST_V   = 0.0639

with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan(
        DEV_CH,
        min_val=-10.0, max_val=10.0,
        terminal_config=TERMINAL
    )
    task.timing.cfg_samp_clk_timing(FS, sample_mode=AcquisitionType.CONTINUOUS)

    print("Read Data (Ctrl+C to shut down)")
    while True:
        vals = task.read(number_of_samples_per_channel=N_PER)
        mean_v = sum(vals)/len(vals)
        forceN = mean_v * CAL - REST_V * CAL
        print(f"{mean_v:.5f} V  →  {forceN:.3f} N")


Read Data (Ctrl+C to shut down)
0.06386 V  →  -0.001 N
0.06381 V  →  -0.002 N
0.06387 V  →  -0.001 N
0.12040 V  →  1.147 N
0.17970 V  →  2.351 N
0.17969 V  →  2.351 N
0.17967 V  →  2.351 N
0.17966 V  →  2.351 N
0.15952 V  →  1.942 N
0.06388 V  →  -0.000 N
0.06403 V  →  0.003 N
0.12987 V  →  1.340 N
0.17320 V  →  2.219 N
0.17324 V  →  2.220 N
0.17326 V  →  2.221 N
0.17324 V  →  2.220 N
0.16890 V  →  2.132 N
0.06388 V  →  -0.000 N
0.06379 V  →  -0.002 N
0.06385 V  →  -0.001 N
0.08765 V  →  0.482 N
0.17009 V  →  2.156 N
0.17292 V  →  2.214 N
0.17279 V  →  2.211 N
0.17285 V  →  2.212 N
0.17296 V  →  2.215 N
0.17288 V  →  2.213 N
0.17292 V  →  2.214 N
0.17288 V  →  2.213 N
0.10687 V  →  0.873 N
0.06389 V  →  -0.000 N
0.06388 V  →  -0.000 N
0.14750 V  →  1.698 N
0.17832 V  →  2.323 N
0.17822 V  →  2.321 N
0.17818 V  →  2.320 N


KeyboardInterrupt: 

In [11]:
print(8.6/35.274*9.80665)
print(8.7/35.274*9.80665)

2.3909165390939497
2.4187178941996934


In [9]:
100/(11.772/2.3909)

20.3100577641862

In [ ]:
# daq_gui.py
# NI USB-6009 load-cell GUI (English UI)
# - Plot centered visually (symmetric padding + balanced subplot margins)
# - Dynamic font scaling buttons (A− / A+)
# - Sampling frequency label split into two lines
# - Gray dashed grid + thicker blue line
# - Safe close (after-cancel + double-call guard)
# - Auto-applied initial axes (X: 0–100 s, Y: -1–50 N)
# - "Logging interval [s]" -> samples_per_read = Fs * interval

import os, csv, time, datetime, threading, queue
import tkinter as tk
from tkinter import ttk, messagebox, font
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

import nidaqmx
from nidaqmx.constants import TerminalConfiguration, AcquisitionType

USB6009_FS_MAX = 48000.0  # guideline single-channel max S/s

class DAQApp:
    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("NI USB-6009 Force Logger (Python)")
        self._closing = False
        self._after_id = None

        # ---------- Font system (dynamic) ----------
        self.base_size = 16
        self.FONT_NORM = font.Font(family="Arial", size=self.base_size)
        self.FONT_BIG  = font.Font(family="Arial", size=self.base_size, weight="bold")
        self.FONT_BTN  = font.Font(family="Arial", size=self.base_size, weight="bold")

        self.style = ttk.Style()
        try:
            self.style.theme_use("clam")
        except Exception:
            pass
        self.style.configure("Big.TButton", font=self.FONT_BTN, padding=14)
        self.style.configure("TCombobox",  font=self.FONT_NORM)
        self._apply_combobox_list_font()

        # ---------- State ----------
        self.running = False
        self.recording = False
        self.reader_stop = False
        self.task = None
        self.csv_writer = None
        self.csv_file = None
        self.t0 = None

        self.tare_v = 0.0
        self.cal = 100.0  # N/V
        self.q = queue.Queue(maxsize=64)
        self.t_data, self.f_data = [], []

        # ---------- Layout ----------==================================================================================================
        # Top controls frame
        frm = ttk.Frame(root, padding=20)
        frm.grid(row=0, column=0, sticky="nsew")
        # Plot frame with symmetric horizontal padding so the canvas looks centered
        self.plot_frame = ttk.Frame(root, padding=(20, 0, 20, 20))  # (left, top, right, bottom)
        self.plot_frame.grid(row=1, column=0, sticky="nsew")

        root.columnconfigure(0, weight=1)
        root.rowconfigure(1, weight=1)  # plot row expands

        PADX, PADY = 16, 10

        # Row 0: Device / Terminal / Voltage range + font scale buttons
        tk.Label(frm, text="Device/Channel", font=self.FONT_BIG)\
            .grid(row=0, column=0, sticky="w", padx=PADX, pady=PADY)
        self.e_chan = tk.Entry(frm, width=16, font=self.FONT_NORM)
        self.e_chan.insert(0, "Dev1/ai0")
        self.e_chan.grid(row=0, column=1, padx=PADX, pady=PADY)

        tk.Label(frm, text="Terminal", font=self.FONT_BIG)\
            .grid(row=0, column=2, sticky="e", padx=PADX, pady=PADY)
        self.cmb_term = ttk.Combobox(frm, width=10, values=["DIFF", "RSE"], state="readonly")
        self.cmb_term.set("DIFF")
        self.cmb_term.grid(row=0, column=3, padx=PADX, pady=PADY)

        tk.Label(frm, text="Voltage range [V]", font=self.FONT_BIG)\
            .grid(row=0, column=4, sticky="e", padx=PADX, pady=PADY)
        self.e_vmin = tk.Entry(frm, width=8, font=self.FONT_NORM); self.e_vmin.insert(0, "-10")
        self.e_vmax = tk.Entry(frm, width=8, font=self.FONT_NORM); self.e_vmax.insert(0, "10")
        self.e_vmin.grid(row=0, column=5, padx=PADX, pady=PADY)
        self.e_vmax.grid(row=0, column=6, padx=PADX, pady=PADY)

        # Font scaling buttons (A− / A+)
        self.btn_font_down = ttk.Button(frm, text="A−", style="Big.TButton",
                                        command=lambda: self.adjust_font(-2))
        self.btn_font_up   = ttk.Button(frm, text="A+", style="Big.TButton",
                                        command=lambda: self.adjust_font(+2))
        self.btn_font_down.grid(row=0, column=7, padx=PADX, pady=PADY, sticky="e")
        self.btn_font_up.grid(row=0, column=8, padx=(0, PADX), pady=PADY, sticky="w")

        # Row 1: Sampling frequency / Logging interval / Calibration
        sf_text = f"Sampling frequency [Samples/s]\n(≤ ~{int(USB6009_FS_MAX):,})"
        tk.Label(frm, text=sf_text, font=self.FONT_BIG, justify="left")\
            .grid(row=1, column=0, sticky="w", padx=PADX, pady=PADY)
        self.e_fs = tk.Entry(frm, width=10, font=self.FONT_NORM); self.e_fs.insert(0, "2000")
        self.e_fs.grid(row=1, column=1, padx=PADX, pady=PADY)

        tk.Label(frm, text="Logging interval [s]", font=self.FONT_BIG)\
            .grid(row=1, column=2, sticky="e", padx=PADX, pady=PADY)
        self.e_dt = tk.Entry(frm, width=10, font=self.FONT_NORM); self.e_dt.insert(0, "0.1")
        self.e_dt.grid(row=1, column=3, padx=PADX, pady=PADY)

        tk.Label(frm, text="Calibration (N/V)", font=self.FONT_BIG)\
            .grid(row=1, column=4, sticky="e", padx=PADX, pady=PADY)
        self.e_cal = tk.Entry(frm, width=10, font=self.FONT_NORM); self.e_cal.insert(0, str(self.cal))
        self.e_cal.grid(row=1, column=5, padx=PADX, pady=PADY)

        self.btn_apply = ttk.Button(frm, text="Apply", style="Big.TButton", command=self.apply_params)
        self.btn_apply.grid(row=1, column=6, padx=PADX, pady=PADY)

        # Row 2: X/Y ranges (manual zoom)
        tk.Label(frm, text="X Min [s]", font=self.FONT_BIG)\
            .grid(row=2, column=0, sticky="e", padx=PADX, pady=PADY)
        self.e_xmin = tk.Entry(frm, width=8, font=self.FONT_NORM); self.e_xmin.insert(0, "0")
        self.e_xmin.grid(row=2, column=1, padx=PADX, pady=PADY)

        tk.Label(frm, text="X Max [s]", font=self.FONT_BIG)\
            .grid(row=2, column=2, sticky="e", padx=PADX, pady=PADY)
        self.e_xmax = tk.Entry(frm, width=8, font=self.FONT_NORM); self.e_xmax.insert(0, "100")
        self.e_xmax.grid(row=2, column=3, padx=PADX, pady=PADY)

        self.btn_applyx = ttk.Button(frm, text="Apply X Range", style="Big.TButton",
                                     command=self.apply_xrange)
        self.btn_applyx.grid(row=2, column=4, padx=PADX, pady=PADY)

        tk.Label(frm, text="Y Min [N]", font=self.FONT_BIG)\
            .grid(row=2, column=5, sticky="e", padx=PADX, pady=PADY)
        self.e_ymin = tk.Entry(frm, width=8, font=self.FONT_NORM); self.e_ymin.insert(0, "-1")
        self.e_ymin.grid(row=2, column=6, padx=PADX, pady=PADY)

        tk.Label(frm, text="Y Max [N]", font=self.FONT_BIG)\
            .grid(row=2, column=7, sticky="e", padx=PADX, pady=PADY)
        self.e_ymax = tk.Entry(frm, width=8, font=self.FONT_NORM); self.e_ymax.insert(0, "50")
        self.e_ymax.grid(row=2, column=8, padx=PADX, pady=PADY)

        self.btn_applyy = ttk.Button(frm, text="Apply Y Range", style="Big.TButton",
                                     command=self.apply_yrange)
        self.btn_applyy.grid(row=2, column=9, padx=PADX, pady=PADY)

        # Row 3: Start/Stop/Tare/Record + CSV path
        self.btn_start = ttk.Button(frm, text="Start", style="Big.TButton", command=self.start)
        self.btn_start.grid(row=3, column=0, pady=PADY, sticky="ew")
        self.btn_stop  = ttk.Button(frm, text="Stop", style="Big.TButton",
                                    command=self.stop, state="disabled")
        self.btn_stop.grid(row=3, column=1, sticky="ew", pady=PADY)
        self.btn_tare  = ttk.Button(frm, text="Tare (Zero)", style="Big.TButton",
                                    command=self.do_tare, state="disabled")
        self.btn_tare.grid(row=3, column=2, sticky="ew", pady=PADY)
        self.btn_rec   = ttk.Button(frm, text="Record: OFF", style="Big.TButton",
                                    command=self.toggle_record, state="disabled")
        self.btn_rec.grid(row=3, column=3, sticky="ew", pady=PADY)

        tk.Label(frm, text="CSV:", font=self.FONT_BIG)\
            .grid(row=3, column=4, sticky="e", padx=PADX, pady=PADY)
        self.lbl_csv = tk.Label(frm, text="(not recording)", font=self.FONT_NORM, anchor="w")
        self.lbl_csv.grid(row=3, column=5, columnspan=4, sticky="w", padx=PADX, pady=PADY)

        # ---- Plot ----
        # Use constrained_layout to help centering within figure boundaries,
        # and also apply manual margins for balanced look.
        self.fig = plt.figure(figsize=(11, 5), dpi=100, constrained_layout=True)
        self.ax = self.fig.add_subplot(111)
        self.ax.set_xlabel("Time [s]", fontsize=16)
        self.ax.set_ylabel("Force [N]", fontsize=16)
        self.ax.tick_params(axis='both', labelsize=13)
        self.ax.grid(True, which="both", color="gray", linestyle="--", linewidth=0.6, alpha=0.5)
        self.line, = self.ax.plot([], [], color="blue", linewidth=2.2)

        # Initial axes
        try:
            self.ax.set_xlim(float(self.e_xmin.get()), float(self.e_xmax.get()))
            self.ax.set_ylim(float(self.e_ymin.get()), float(self.e_ymax.get()))
        except Exception:
            self.ax.set_xlim(0, 100)
            self.ax.set_ylim(-1, 50)

        # Canvas inside the centered plot_frame (with symmetric padding)
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.plot_frame)
        self.canvas.get_tk_widget().grid(row=0, column=0, sticky="nsew")
        self.plot_frame.columnconfigure(0, weight=1)
        self.plot_frame.rowconfigure(0, weight=1)
        self.canvas.draw_idle()

        # Safe close
        root.protocol("WM_DELETE_WINDOW", self.on_close)
        # Periodic update
        self._schedule_update()

    #================================================================================================================

    # ---------- Internal helpers ----------
    def _apply_combobox_list_font(self):
        # Enlarge dropdown list font as well
        self.root.option_add("*TCombobox*Listbox*Font", ("Arial", 16))

    def _schedule_update(self):
        self._after_id = self.root.after(80, self.update_plot)

    def adjust_font(self, delta: int):
        # Update base size and apply to all font objects
        new_size = max(8, self.base_size + delta)
        if new_size == self.base_size:
            return
        self.base_size = new_size
        for f in (self.FONT_NORM, self.FONT_BIG, self.FONT_BTN):
            f.configure(size=self.base_size)

        # Re-apply combobox list font
        self._apply_combobox_list_font()

        # Update plot tick/label font sizes too
        self.ax.set_xlabel("Time [s]", fontsize=self.base_size + 0)
        self.ax.set_ylabel("Force [N]", fontsize=self.base_size + 0)
        self.ax.tick_params(axis='both', labelsize=max(8, self.base_size - 3))
        self.canvas.draw_idle()

    # ---------- Handlers ----------
    def apply_params(self):
        try:
            self.cal = float(self.e_cal.get())

            fs = float(self.e_fs.get())
            if fs <= 0:
                raise ValueError("Sampling frequency must be > 0.")
            if fs > USB6009_FS_MAX:
                messagebox.showwarning(
                    "Fs capped",
                    f"Sampling frequency exceeds device guideline (~{int(USB6009_FS_MAX):,} S/s). "
                    f"Capping to {int(USB6009_FS_MAX)}."
                )
                fs = USB6009_FS_MAX
                self.e_fs.delete(0, tk.END); self.e_fs.insert(0, str(int(fs)))

            dt = float(self.e_dt.get())
            if dt <= 0:
                raise ValueError("Logging interval must be > 0 s.")

            if self.running:
                messagebox.showinfo("Info", "Stop acquisition before applying parameters.")
        except Exception as e:
            messagebox.showerror("Parameter error", str(e))

    def apply_xrange(self):
        try:
            xmin = float(self.e_xmin.get())
            xmax = float(self.e_xmax.get())
            if xmin >= xmax:
                raise ValueError("X Min must be < X Max.")
            self.ax.set_xlim(xmin, xmax)
            self.canvas.draw_idle()
        except Exception as e:
            messagebox.showerror("X Range error", str(e))

    def apply_yrange(self):
        try:
            ymin = float(self.e_ymin.get())
            ymax = float(self.e_ymax.get())
            if ymin >= ymax:
                raise ValueError("Y Min must be < Y Max.")
            self.ax.set_ylim(ymin, ymax)
            self.canvas.draw_idle()
        except Exception as e:
            messagebox.showerror("Y Range error", str(e))

    def start(self):
        if self.running:
            return
        try:
            chan = self.e_chan.get().strip()

            fs = float(self.e_fs.get())
            if fs <= 0:
                raise ValueError("Sampling frequency must be > 0.")
            if fs > USB6009_FS_MAX:
                messagebox.showwarning(
                    "Fs capped",
                    f"Sampling frequency exceeds device guideline (~{int(USB6009_FS_MAX):,} S/s). "
                    f"Capping to {int(USB6009_FS_MAX)}."
                )
                fs = USB6009_FS_MAX
                self.e_fs.delete(0, tk.END); self.e_fs.insert(0, str(int(fs)))

            dt = float(self.e_dt.get())
            if dt <= 0:
                raise ValueError("Logging interval must be > 0 s.")

            nper = max(1, int(round(fs * dt)))  # samples per read

            vmin = float(self.e_vmin.get())
            vmax = float(self.e_vmax.get())
            term = TerminalConfiguration.DIFF if self.cmb_term.get() == "DIFF" \
                   else TerminalConfiguration.RSE
        except Exception as e:
            messagebox.showerror("Parameter error", str(e))
            return

        self.running = True
        self.recording = False
        self.reader_stop = False
        self.btn_start.config(state="disabled")
        self.btn_stop.config(state="normal")
        self.btn_tare.config(state="normal")
        self.btn_rec.config(state="normal", text="Record: OFF")
        self.lbl_csv.config(text="(not recording)")

        self.t_data.clear()
        self.f_data.clear()
        self.t0 = time.time()
        self.tare_v = 0.0

        # DAQ task
        self.task = nidaqmx.Task()
        self.task.ai_channels.add_ai_voltage_chan(
            chan, min_val=vmin, max_val=vmax, terminal_config=term
        )
        self.task.timing.cfg_samp_clk_timing(rate=fs, sample_mode=AcquisitionType.CONTINUOUS)

        # Background reader
        self.reader = threading.Thread(target=self._reader_loop, args=(nper, fs), daemon=True)
        self.reader.start()

    def stop(self):
        # stop recording
        self.recording = False
        self.btn_rec.config(text="Record: OFF")
        if self.csv_file:
            try:
                self.csv_file.close()
            except Exception:
                pass
            self.csv_writer = None
            self.csv_file = None
        self.lbl_csv.config(text="(not recording)")

        # stop reader/task
        self.reader_stop = True
        if self.task:
            try:
                self.task.close()
            except Exception:
                pass
            self.task = None

        self.running = False
        self.btn_start.config(state="normal")
        self.btn_stop.config(state="disabled")
        self.btn_tare.config(state="disabled")
        self.btn_rec.config(state="disabled")

    def do_tare(self):
        """Make the recent mean force go to zero by adjusting voltage offset."""
        if not self.f_data:
            self.tare_v = 0.0
            return
        window = self.f_data[-50:] if len(self.f_data) >= 50 else self.f_data
        mean_force = sum(window) / len(window) if window else 0.0
        self.tare_v += (mean_force / self.cal)  # force=(v-tare)*cal -> tare += mean/cal

    def toggle_record(self):
        if not self.running:
            return
        self.recording = not self.recording
        if self.recording:
            ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            fname = f"daq_{ts}.csv"
            self.csv_file = open(fname, "w", newline="")
            self.csv_writer = csv.writer(self.csv_file)
            self.csv_writer.writerow(["t_sec", "voltage_V", "force_N"])
            self.lbl_csv.config(text=os.path.abspath(fname))
            self.btn_rec.config(text="Record: ON")
        else:
            if self.csv_file:
                try:
                    self.csv_file.close()
                except Exception:
                    pass
            self.csv_writer = None
            self.csv_file = None
            self.lbl_csv.config(text="(not recording)")
            self.btn_rec.config(text="Record: OFF")

    # ---------- Background reader ----------
    def _reader_loop(self, nper: int, fs: float):
        try:
            while not self.reader_stop:
                vals = self.task.read(number_of_samples_per_channel=nper)
                t_now = time.time() - self.t0
                v_mean = sum(vals) / len(vals) if vals else 0.0
                try:
                    self.q.put_nowait((t_now, v_mean))
                except queue.Full:
                    pass
                # small sleep (~5% of the logging interval) to reduce UI pressure
                interval = nper / fs if fs > 0 else 0.1
                time.sleep(max(0.0, interval * 0.05))
        except Exception as e:
            try:
                self.q.put(("__error__", str(e)))
            except Exception:
                pass

    # ---------- UI updater ----------
    def update_plot(self):
        while True:
            try:
                item = self.q.get_nowait()
            except queue.Empty:
                break

            if item[0] == "__error__":
                messagebox.showerror("DAQ error", item[1])
                self.stop()
                break

            t, v = item
            force = (v - self.tare_v) * self.cal
            self.t_data.append(t)
            self.f_data.append(force)

            if self.recording and self.csv_writer:
                self.csv_writer.writerow([f"{t:.6f}", f"{v:.6f}", f"{force:.6f}"])

        if self.t_data:
            self.line.set_data(self.t_data, self.f_data)
            # keep tick font in sync with UI size
            self.ax.tick_params(axis='both', labelsize=max(8, self.base_size - 3))
            self.canvas.draw_idle()

        self._schedule_update()

    # ---------- Safe close ----------
    def on_close(self):
        if self._closing:
            return
        self._closing = True

        try:
            if self.running:
                self.stop()
        except Exception:
            pass

        if self._after_id is not None:
            try:
                self.root.after_cancel(self._after_id)
            except Exception:
                pass
            self._after_id = None

        try:
            if self.root.winfo_exists():
                self.root.destroy()
        except Exception:
            pass


if __name__ == "__main__":
    root = tk.Tk()
    app = DAQApp(root)
    root.mainloop()


Run below code block (console command) to compile thrust_stand.py into Executable file.

```!pyinstaller --onefile --noconsole --copy-metadata nidaqmx --copy-metadata nitypes --name NI_USB6009_ForceLogger thrust_stand.py```


In [2]:
!pyinstaller --onefile --noconsole --copy-metadata nidaqmx --copy-metadata nitypes --name NI_USB6009_ForceLogger thrust_stand.py

987 INFO: PyInstaller: 6.16.0, contrib hooks: 2025.9
987 INFO: Python: 3.11.14 (conda)
1002 INFO: Platform: Windows-10-10.0.26200-SP0
1002 INFO: Python environment: C:\Users\eugene\AppData\Local\miniconda3\envs\ae100pypi
1004 INFO: wrote c:\Users\eugene\Documents\School\Fall_2025\ae_100\thrust stand demo code\NI_USB6009_ForceLogger.spec
1031 INFO: Module search paths (PYTHONPATH):
['C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100pypi\\Scripts\\pyinstaller.exe',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100pypi\\python311.zip',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100pypi\\DLLs',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100pypi\\Lib',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100pypi',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100pypi\\Lib\\site-packages',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100pypi\\Lib\\site-packages\\setuptools\\_vendor',
 'c:\\Users\\eugene\\Documents\\Sc

This code block will include ALL dependencies of nidaqmx, including stuff not needed. This can produce enormous file sizes (4x what is actually needed). Only use this if the above block doesn't work.

In [4]:
!pyinstaller --onefile --noconsole --recursive-copy-metadata nidaqmx --name NI_USB6009_ForceLogger thrust_stand.py

12028 INFO: PyInstaller: 6.16.0, contrib hooks: 2025.9
12028 INFO: Python: 3.10.19 (conda)
12052 INFO: Platform: Windows-10-10.0.26200-SP0
12052 INFO: Python environment: C:\Users\eugene\AppData\Local\miniconda3\envs\ae100
12058 INFO: wrote c:\Users\eugene\Documents\School\Fall_2025\ae_100\thrust stand demo code\NI_USB6009_ForceLogger.spec
12697 INFO: Module search paths (PYTHONPATH):
['C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100\\Scripts\\pyinstaller.exe',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100\\python310.zip',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100\\DLLs',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100\\lib',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100\\lib\\site-packages',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100\\lib\\site-packages\\win32',
 'C:\\Users\\eugene\\AppData\\Local\\miniconda3\\envs\\ae100\\lib\\site-